In [2]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [3]:
data_path = os.path.join("model_data")

actions = []

for i in range(65,91):
    actions.append(chr(i))

sequences_no = 60

sequences_length = 30

In [4]:
actions = np.array(actions)

In [39]:
# actions = np.array(['A','B'])

In [5]:
actions

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
      dtype='<U1')

In [6]:
data_path

'model_data'

In [7]:
label_map = {}
i = 0
for num in actions:
    label_map[num] = i
    i += 1


In [8]:
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in range(sequences_no):
        window = []
        for frame_num in range(sequences_length):
            path = os.path.join(data_path,str(action),str(sequence),"{}.npy".format(frame_num))
            res = np.load(path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
X = np.array(sequences)

In [10]:
X.shape

(1560, 30, 126)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [14]:
X_train

array([[[ 5.13902724e-01,  8.53825867e-01,  1.14813695e-07, ...,
          4.43333685e-01,  5.53503811e-01, -3.00453827e-02],
        [ 5.06553411e-01,  8.41748953e-01,  1.06437476e-07, ...,
          4.43795800e-01,  5.54241598e-01, -3.10338158e-02],
        [ 5.14262319e-01,  8.52889001e-01,  1.20639157e-07, ...,
          4.43781465e-01,  5.53961515e-01, -3.01671047e-02],
        ...,
        [ 5.07010341e-01,  8.41126502e-01,  9.29827166e-08, ...,
          4.43924040e-01,  5.53811431e-01, -3.08609735e-02],
        [ 5.13924718e-01,  8.54253173e-01,  1.06784903e-07, ...,
          4.43550587e-01,  5.53014278e-01, -2.99713034e-02],
        [ 5.07203043e-01,  8.41370046e-01,  9.46076355e-08, ...,
          4.43871886e-01,  5.53317785e-01, -3.08654383e-02]],

       [[ 6.52839780e-01,  6.29672945e-01, -1.55576345e-07, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 6.52846813e-01,  6.29794300e-01, -1.56360727e-07, ...,
          0.00000000e+00,  0.00000000e

Traning Neural Network (LSTM)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [61]:
model = Sequential()

model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(BatchNormalization())

model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=False, activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))

model.add(Dense(32, activation = 'relu'))

model.add(Dense(actions.shape[0], activation='softmax'))

c:\Users\agarw\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [49]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)

In [50]:
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['categorical_accuracy'])

In [62]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 30, 128)        │       130,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 30, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 30, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 30, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 751,162 (2.87 MB)

 Trainable params: 750,394 (2.86 MB)

 Non-trainable params: 768 (3.00 KB)

In [52]:
model.fit(X_train,y_train, epochs=20)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - categorical_accuracy: 0.0760 - loss: 3.2509
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - categorical_accuracy: 0.1414 - loss: 3.0899
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - categorical_accuracy: 0.3073 - loss: 2.4645
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - categorical_accuracy: 0.4745 - loss: 1.9250
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - categorical_accuracy: 0.5547 - loss: 1.5051
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - categorical_accuracy: 0.5289 - loss: 1.6345
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - categorical_accuracy: 0.6854 - loss: 1.1085
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - categorical_accuracy: 0.6036 - loss: 1.3306
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - categorical_accuracy: 0.8451 - loss: 0.5611
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - categorical_accuracy: 0.8657 - loss: 0.5646
Epoch 11/100
44/4

KeyboardInterrupt: 

In [53]:
res = model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step


In [54]:
pred = []
for i in res:
    print(actions[np.argmax(i)],end=" ")
    pred.append(np.argmax(i))

Z W C G J K P R C I I J O I T S I R W K I B M F G C U L M I E Q D G T U X J P Q N G B D V T L X G U C W S R V H Y K Y U F Q C D E D P T H K H S Z R Z M V Z R Q O M W R O U Y L T I N E O R V U Q T O A R G X A S B T L C B X O A T U L L I U M F O X C C S G D E J N J Y V B A N C Z E R U V Q B I U D J P K S Q M L B 

In [55]:
test = []
for i in y_test:
    print(actions[np.argmax(i)],end=" ")
    test.append(np.argmax(i))

Z W C G J K P R C I I J O I T S I R W K I B M F G C U L M I E Q D G T U X J P Q N G B D V T L X G U C W S R V H Y K Y U F Q C D E D P T H K H S Z R Z M V Z R Q O M W R O U Y L T I N E O R V U Q T O A R G X A S B T L C B X O A T U L L I U M F O X C C S G D E J N J Y V B A N C Z E R U V Q B I U D J P K S Q M L B 

In [60]:
model.save('modek.keras')

In [56]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,f1_score,recall_score

In [57]:
y_true = np.argmax(y_test,axis=1).tolist()
y_predic = np.argmax(res,axis=1).tolist()

print("Actual values:",y_true)
print("Predicted Value:",y_predic)

Actual values: [25, 22, 2, 6, 9, 10, 15, 17, 2, 8, 8, 9, 14, 8, 19, 18, 8, 17, 22, 10, 8, 1, 12, 5, 6, 2, 20, 11, 12, 8, 4, 16, 3, 6, 19, 20, 23, 9, 15, 16, 13, 6, 1, 3, 21, 19, 11, 23, 6, 20, 2, 22, 18, 17, 21, 7, 24, 10, 24, 20, 5, 16, 2, 3, 4, 3, 15, 19, 7, 10, 7, 18, 25, 17, 25, 12, 21, 25, 17, 16, 14, 12, 22, 17, 14, 20, 24, 11, 19, 8, 13, 4, 14, 17, 21, 20, 16, 19, 14, 0, 17, 6, 23, 0, 18, 1, 19, 11, 2, 1, 23, 14, 0, 19, 20, 11, 11, 8, 20, 12, 5, 14, 23, 2, 2, 18, 6, 3, 4, 9, 13, 9, 24, 21, 1, 0, 13, 2, 25, 4, 17, 20, 21, 16, 1, 8, 20, 3, 9, 15, 10, 18, 16, 12, 11, 1]
Predicted Value: [25, 22, 2, 6, 9, 10, 15, 17, 2, 8, 8, 9, 14, 8, 19, 18, 8, 17, 22, 10, 8, 1, 12, 5, 6, 2, 20, 11, 12, 8, 4, 16, 3, 6, 19, 20, 23, 9, 15, 16, 13, 6, 1, 3, 21, 19, 11, 23, 6, 20, 2, 22, 18, 17, 21, 7, 24, 10, 24, 20, 5, 16, 2, 3, 4, 3, 15, 19, 7, 10, 7, 18, 25, 17, 25, 12, 21, 25, 17, 16, 14, 12, 22, 17, 14, 20, 24, 11, 19, 8, 13, 4, 14, 17, 21, 20, 16, 19, 14, 0, 17, 6, 23, 0, 18, 1, 19, 11, 2, 1, 2

In [58]:
multilabel_confusion_matrix(y_true,y_predic)

array([[[152,   0],
        [  0,   4]],

       [[149,   0],
        [  0,   7]],

       [[147,   0],
        [  0,   9]],

       [[150,   0],
        [  0,   6]],

       [[151,   0],
        [  0,   5]],

       [[153,   0],
        [  0,   3]],

       [[149,   0],
        [  0,   7]],

       [[153,   0],
        [  0,   3]],

       [[147,   0],
        [  0,   9]],

       [[150,   0],
        [  0,   6]],

       [[151,   0],
        [  0,   5]],

       [[149,   0],
        [  0,   7]],

       [[150,   0],
        [  0,   6]],

       [[152,   0],
        [  0,   4]],

       [[149,   0],
        [  0,   7]],

       [[152,   0],
        [  0,   4]],

       [[149,   0],
        [  0,   7]],

       [[147,   0],
        [  0,   9]],

       [[150,   0],
        [  0,   6]],

       [[148,   0],
        [  0,   8]],

       [[146,   0],
        [  0,  10]],

       [[150,   0],
        [  0,   6]],

       [[152,   0],
        [  0,   4]],

       [[151,   0],
        [  0, 

In [59]:
accuracy_score(y_true,y_predic)

1.0